# 코로나 확진자 수 얻기

> 피드백 환영



In [ ]:
!pip install xmltodict

### CSV파일 추출하기

In [ ]:
# csv파일 추출 함수 작성

### 데이터프레임으로 변환하기

In [ ]:
# 데이터프레임 변환 함수 작성

## 전국 코로나 확진자 수
공공 데이터 포털 API 이용 [링크](https://www.data.go.kr/iim/api/selectAPIAcountView.do)

In [ ]:
import pandas as pd
import requests
import json
import xmltodict
import datetime
from bs4 import BeautifulSoup

ServiceKey = 'rC%2BmF3DCdE%2BCfEcFI9xIb%2FIzswvPz1MPV5rx4MoHur2mLbP4HAple5n8itK18TW9PwYMlpTIoArMpyA4DN8oYQ%3D%3D' # 키
pageNo = '1'
numOfRows = '10'
startCreateDt = '20200101'  # 검색할 생성일 범위의 시작
endCreateDt = '20210707'  # 검색할 생성일 범위의 종료

ROOT_URL = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson?"

REQ_URL = ROOT_URL + 'serviceKey=' + ServiceKey + '&pageNo=' + pageNo + '&numOfRows' + numOfRows + '&startCreateDt=' + startCreateDt + '&endCreateDt=' + endCreateDt
# print(REQ_URL)
response = requests.get(REQ_URL)
if response.status_code == 200:
    xmlString = response.text
    jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)
    jsonString = jsonString.replace('resultCode', '결과코드').replace('resultMsg', '결과메세지').replace('numOfRows',
                                                                                                '한 페이지 결과 수').replace(
        'pageNo', '페이지 수').replace('totalCount', '전체 결과 수').replace('seq', '게시글번호(감염현황 고유값)').replace('stateDt',
                                                                                                      '기준일').replace(
        'stateTime', '기준시간').replace('decideCnt', '확진자 수').replace('clearCnt', '격리해제 수').replace('examCnt',
                                                                                                  '검사진행 수').replace(
        'deathCnt', '사망자 수').replace('careCnt', '치료중 환자 수').replace('resutlNegCnt', '결과 음성 수').replace('accExamCnt',
                                                                                                        '누적 검사 수').replace(
        'accExamCompCnt', '누적 검사 완료 수').replace('accDefRate', '누적 환진률').replace('createDt', '등록일시분초').replace(
        'updateDt', '수정일시분초')
    print(jsonString)

{
    "response": {
        "header": {
            "결과코드": "00",
            "결과메세지": "NORMAL SERVICE."
        },
        "body": {
            "items": {
                "item": [
                    {
                        "누적 환진률": "1.5319725965",
                        "누적 검사 수": "10766502",
                        "누적 검사 완료 수": "10623754",
                        "치료중 환자 수": "9220",
                        "격리해제 수": "151500",
                        "등록일시분초": "2021-07-07 09:41:09.863",
                        "사망자 수": "2033",
                        "확진자 수": "162753",
                        "검사진행 수": "142748",
                        "결과 음성 수": "10461001",
                        "게시글번호(감염현황 고유값)": "565",
                        "기준일": "20210707",
                        "기준시간": "00:00",
                        "수정일시분초": "null"
                    },
                    {
                        "누적 환진률": "1.5248323652",
                        "누적 검사 수": "10735716",
       

In [ ]:
data_dict = json.loads(jsonString)
date = []
decide = []
all_items = data_dict["response"]['body']['items']['item']
for i in range(1, len(all_items)):
    decide_cnt = int(all_items[i - 1]['확진자 수']) - int(all_items[i]['확진자 수'])
    date.insert(0, all_items[i]['기준일'])
    decide.insert(0, decide_cnt)

# print(date)
# print(decide)

df = pd.DataFrame(data=list(zip(date, decide)), columns=['날짜', '확진자 수'])
df['날짜'] = pd.to_datetime(df['날짜'])
print(df)

            날짜  확진자 수
0   2020-01-01      0
1   2020-02-01      2
2   2020-02-02     13
3   2020-02-03    -15
4   2020-02-04     19
..         ...    ...
518 2021-07-02    794
519 2021-07-03    742
520 2021-07-04    711
521 2021-07-05    746
522 2021-07-06   1212

[523 rows x 2 columns]


## 지역별 확진자 수
> 각 경기장:지역 맵핑


In [ ]:
import requests
from bs4 import BeautifulSoup

searchStartDate = '2020.01.01'
searchEndDate = '2021.07.07'

### 서울 확진자 수
> 서울잠실야구장에 반영하기

In [ ]:
# 서울시 코로나 확진자
ROOT_URL = "https://www.seoul.go.kr/coronaV/searchCoronaDayStatus.do?"

REQ_URL = ROOT_URL + 'sdate=' + searchStartDate + '&edate=' + searchEndDate
# print(REQ_URL)
response = requests.get(REQ_URL)
if response.status_code == 200:
    text = response.json()
    # print(text)
    for eachDay in text['rVal']:
        eachDate = (datetime.datetime.strptime(eachDay['C_DATE2'], '%Y.%m.%d') - datetime.timedelta(days=1)).date()
        eachGap = eachDay['GAP']
        print(f'날짜: {eachDate} 확진자 수: {eachGap}')

날짜: 2020-01-24 확진자 수: 0
날짜: 2020-01-25 확진자 수: 0
날짜: 2020-01-26 확진자 수: 0
날짜: 2020-01-27 확진자 수: 0
날짜: 2020-01-28 확진자 수: 0
날짜: 2020-01-29 확진자 수: 0
날짜: 2020-01-30 확진자 수: 3
날짜: 2020-01-31 확진자 수: 3
날짜: 2020-02-01 확진자 수: 0
날짜: 2020-02-02 확진자 수: 1
날짜: 2020-02-03 확진자 수: 0
날짜: 2020-02-04 확진자 수: 0
날짜: 2020-02-05 확진자 수: 2
날짜: 2020-02-06 확진자 수: 2
날짜: 2020-02-07 확진자 수: 0
날짜: 2020-02-08 확진자 수: 0
날짜: 2020-02-09 확진자 수: 0
날짜: 2020-02-10 확진자 수: 0
날짜: 2020-02-11 확진자 수: 0
날짜: 2020-02-12 확진자 수: 0
날짜: 2020-02-13 확진자 수: 0
날짜: 2020-02-14 확진자 수: 0
날짜: 2020-02-15 확진자 수: 0
날짜: 2020-02-16 확진자 수: 2
날짜: 2020-02-17 확진자 수: 0
날짜: 2020-02-18 확진자 수: 0
날짜: 2020-02-19 확진자 수: 2
날짜: 2020-02-20 확진자 수: 5
날짜: 2020-02-21 확진자 수: 2
날짜: 2020-02-22 확진자 수: 5
날짜: 2020-02-23 확진자 수: 3
날짜: 2020-02-24 확진자 수: 4
날짜: 2020-02-25 확진자 수: 16
날짜: 2020-02-26 확진자 수: 10
날짜: 2020-02-27 확진자 수: 9
날짜: 2020-02-28 확진자 수: 12
날짜: 2020-02-29 확진자 수: 5
날짜: 2020-03-01 확진자 수: 5
날짜: 2020-03-02 확진자 수: 6
날짜: 2020-03-03 확진자 수: 0
날짜: 2020-03-04 확진자 수: 7
날짜: 2020-03-0